# WSI
## Raport z ćwiczenia laboratoryjnego nr 5
### Kacper Bugała, 303739 i Jan Kuc, 303753

### 1. Cel eksperymentów

Zadanie polega na implementacji perceptronu wielowarstwowego oraz wybrangeo algorytmu optymalizacji gradientowej z algorytem propagacji wstecznej. Następnie nalezy wytrenować stworzoną sieć do klasyfikacji odręcznie pisanych cyfr ze zbioru MNIST oraz sprawdzić efekty działania.

### 2. Opis eksperymentów i decyzje projektowe

Do implementacji perceptronu wielowarstwowego stworzono uniwersalną klasę **Network**, która korzysta z pomocniczej klasy **Layer**, dzięki czemu mozna zdefiniować dowolną liczbę warstw ukrytych sieci, podając równiez funkcję aktywacji oraz liczbę neuronów w warstwach ukrytych.

Ostatecznie zdecydowano się na trening oraz testowanie sieci składającej się z 2 warstw ukrytych aktywowanych funkcją *ReLu*. Funkcją aktywacji warstwy wyjściowej jest *softmax*. Funkcja ta, doskonale nadaje się do zadań klasyfikacji wieloklasowej i jest potrzebna do normalizacji wyników na końcu sieci. Dzięki niej na wyjściu otrzymujemy tak naprawdę głosowanie modelu na kazdą z mozliwych klas, a więc ocenione przez model prawdopodobieństwo przynalezności podanego przykładu wejściowego do danej klasy.

Jako algorytm opytmalizacji gradientowej wybrano SGD (Stochastic Gradient Descent). Trening modelu podzielony jest na określoną w parametrach wejśiowych liczbę **epok**, a w skład kazdej epoki wchodzą iteracje, których liczba jest zalezna od podanego w parametrach rozmiaru paczki, tzw. **batch size**.
Metoda podziału danych na losowe paczki w trakcie uczenia ma na celu przyspieszenie uczenia, poniewaz w takim przypadku w kazdej iteracji przez siec "przechodzi" tylko mały fragment danych wejściowych.